<a href="https://colab.research.google.com/github/MsMc24/G1-finalproject/blob/main/synonym_matching_MiranCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synonym matching application

## [1] Original code from Megan

=> To make an interactive application, you always need either 'graio' or similar libraries. See the code below for my final implementation.

In [1]:
# Import libraries
from ipywidgets import Text, Button, VBox

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success" ]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

# Initialize variables
score = 0
current_question = 0

# Define functions
def display_question():
    global current_question
    question_widget = Text(description=questions[current_question])
    check_button = Button(description="Check Answer")

    def check_answer(b):
        global current_question, score
        if question_widget.value.lower() == answers[current_question].lower():
            score += 1
            print("Correct!")
        else:
            print("Incorrect. The correct answer is", answers[current_question])
        current_question += 1
        if current_question < len(questions):
            display_question()
        else:
            print("Check how you did. Your score is", score, "out of", len(questions))

    check_button.on_click(check_answer)

    display(VBox([question_widget, check_button]))

# Let's match synonyms
display_question()

Correct!


# Miran versioin

1. To test on Clab, you need to install 'graio'
2. I revised the code to show 4 options so that the learner can choose one of them, but still practice typing.
3. The final score will appear when the user click 'Complete and see the score'
4. If the user try the same one

In [ ]:
%%capture
!pip install gradio

# this works!

# Miran's message: This is the final code. You don't need requirements.txt file for this application because it only uses 'gradio' which is set when you create a Space.

# This is the final code

In [55]:
import gradio as gr
import random

# Define questions and answers
questions = ["colorful", "worried", "excited", "showing", "respected", "doubters", "success"]
answers = ["vibrant", "anxious", "thrilled", "depicting", "admired", "skeptics", "achievement"]

answered_questions = set()  # Track which questions have been correctly answered

def get_options(question):
    index = questions.index(question)
    correct_answer = answers[index]
    options = set([correct_answer])
    while len(options) < 4:
        options.add(random.choice(answers))
    options = list(options)
    random.shuffle(options)
    return ", ".join(options), correct_answer, index

def check_answer(user_input, correct_answer, question_index):
    if question_index in answered_questions:
        return "Correct! You have already answered this question correctly.", 0
    if user_input.strip().lower() == correct_answer.lower():
        feedback = "Correct!"
        score = 1
        answered_questions.add(question_index)  # Only add to answered if correct
    else:
        feedback = f"Incorrect. The correct answer was '{correct_answer}'."
        score = 0
    return feedback, score

with gr.Blocks() as app:
    with gr.Row():
        question_dropdown = gr.Dropdown(choices=questions, label="Select a question")
        show_options_button = gr.Button("Show me options")
    options_output = gr.Textbox(label="Options", interactive=False)
    correct_answer_store = gr.State()
    question_index_store = gr.State()
    user_input = gr.Textbox(label="Type your answer here", visible=True)
    submit_button = gr.Button("Submit Answer", visible=True)
    feedback_label = gr.Label()
    score_store = gr.State(value=0)
    complete_button = gr.Button("Complete and Show Total Score", visible=True)

    def show_options(question):
        options, correct, index = get_options(question)
        options_output.value = options
        correct_answer_store.value = correct
        question_index_store.value = index
        return options, correct, index

    show_options_button.click(
        fn=show_options,
        inputs=[question_dropdown],
        outputs=[options_output, correct_answer_store, question_index_store]
    )

    submit_button.click(
        fn=lambda user_input, correct_answer_store=correct_answer_store, question_index_store=question_index_store, score_store=score_store: submit_response(user_input, correct_answer_store.value, question_index_store.value, score_store),
        inputs=user_input,
        outputs=[feedback_label, score_store]
    )

    def submit_response(user_input, correct_answer, question_index, score_store):
        feedback, score = check_answer(user_input, correct_answer, question_index)
        if score == 1:
            score_store.value += score  # Only accumulate the score if it's a new correct answer
        return feedback, score_store.value

    def complete_and_reset(score_store):
        final_score = score_store.value
        score_store.value = 0  # Reset the score
        answered_questions.clear()  # Clear the set of answered questions
        return f"Session Complete. Your final score is {final_score}."

    complete_button.click(
        fn=lambda score_store=score_store: complete_and_reset(score_store),
        inputs=[],
        outputs=feedback_label
    )

app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b6b5d4a97e93a810ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
